In [1]:
# based on https://www.kaggle.com/jaromiru/keras-cnn-dropout-augmentation-30-min-99-4
# sliced validation data from kaggle training set

import pandas, numpy, time

from collections import namedtuple

from keras import backend as K
K.set_image_dim_ordering('th')

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import *
from keras.preprocessing.image import ImageDataGenerator

from sklearn.utils import shuffle

DATA_DIR = '../input/'
KAGGLE_TRAIN_CSV = 'train.csv'
KAGGLE_TEST_CSV = 'test.csv'
SUBMISSION_FILE = 'submission_mnist_keras_cnn_augmented'

Using Theano backend.
Using cuDNN version 5110 on context None
Mapped name None to device cuda: GeForce GTX 1080 Ti (0000:01:00.0)


In [2]:
Dataset = namedtuple('Dataset', 'x y')

def read_train_valid_test():
    data = pandas.read_csv(DATA_DIR + KAGGLE_TRAIN_CSV)
    data = shuffle(data, random_state=42)

    train_raw = numpy.array(data[:38000])
    valid_raw = numpy.array(data[38000:])

    train_x = train_raw[:,1:].reshape(train_raw.shape[0], 1, 28, 28).astype('float32') / 255.
    train_y = np_utils.to_categorical(train_raw[:,0], 10)

    valid_x = valid_raw[:,1:].reshape(valid_raw.shape[0], 1, 28, 28).astype('float32') / 255.
    valid_y = np_utils.to_categorical(valid_raw[:,0], 10)

    test_raw = numpy.array(pandas.read_csv(DATA_DIR + KAGGLE_TEST_CSV))
    test_x = test_raw.reshape(test_raw.shape[0], 1, 28, 28).astype('float32') / 255.

    return Dataset(x=train_x, y=train_y), Dataset(x=valid_x, y=valid_y), Dataset(x=test_x, y=None)

train, valid, test = read_train_valid_test()

In [3]:
def get_model():
    
    model = Sequential()

    model.add( Convolution2D(32, 5, 5, activation='relu', input_shape=(1, 28, 28)) )
    model.add( MaxPooling2D() )

    model.add( Convolution2D(32, 3, 3, activation='relu') )
    model.add( MaxPooling2D() )

    model.add( Flatten() ) 

    model.add( Dense(output_dim=128, activation='relu') )
    model.add( Dropout(0.5) )

    model.add( Dense(output_dim=10, activation='softmax') )

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    model.summary()
    
    return model

model = get_model()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 32, 24, 24)    832         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 32, 12, 12)    0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 32, 10, 10)    9248        maxpooling2d_1[0][0]             
____________________________________________________________________________________________________
maxpooling2d_2 (MaxPooling2D)    (None, 32, 5, 5)      0           convolution2d_2[0][0]            
___________________________________________________________________________________________

In [4]:
datagen = ImageDataGenerator(
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1
)

In [ ]:
for step in range(20): # change steps to 20
    print('*** Step {0:02d} ***'.format(step+1))
    model.fit_generator(datagen.flow(train.x, train.y, batch_size=64),
        len(train.x), nb_epoch=10, validation_data=(valid.x,valid.y))

    # model.save('model_mnist.h5') # uncomment to save your network

In [6]:
# last epoch:  8s - loss: 0.0245 - acc: 0.9925 - val_loss: 0.0294 - val_acc: 0.9930

In [7]:
y = model.predict_classes(test.x)
filename = SUBMISSION_FILE + "-" + time.strftime("%Y%m%d%H%M%S") + '.csv'
numpy.savetxt(filename, np.c_[range(1,len(y)+1), y], fmt='%d', delimiter=',', header='ImageId,Label')

26688/28000 [===========================>..] - ETA: 0s

In [8]:
# kaggle score: 0.99443